In [ ]:
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip
!unzip -o chinese_L-12_H-768_A-12.zip

Archive:  chinese_L-12_H-768_A-12.zip
   creating: chinese_L-12_H-768_A-12/
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: chinese_L-12_H-768_A-12/vocab.txt  
  inflating: chinese_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: chinese_L-12_H-768_A-12/bert_config.json  


In [ ]:
SEQ_LEN = 512
BATCH_SIZE = 32
EPOCHS = 5

In [ ]:
import os
pretrained_path = 'chinese_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')


# 讀取seg

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import json
import numpy  as np
import pandas as pd
from glob import glob 
from tqdm import tqdm

In [ ]:
df = pd.read_pickle('../content/drive/My Drive/Nomura/strong_data_finaltidy.pickle')

In [ ]:
y = pd.read_csv('../content/drive/My Drive/Nomura/nomura_redemp_taiwan.csv')

In [ ]:
df.head()

,article_index,content,title,time,author,articles_no_author,from,author_identity,author_enroll_time,word_sentence_list,seg,date
0,10,有一檔利安越南基金喔可以試試看,Re: 有人投資過越南基金嗎?,2011-01-01 22:34:00,8d8d,"""725""",NaN,天使人,"""2010-12-10 09:48""","["", 有, 一, 檔, 利安, 越南, 基金, 喔, 可以, 試試, 看, ""]","["", 檔, 利安, 越南, 基金, 喔, 試試, ""]",2011-01-01
1,10,利安越南基金可以在先鋒投顧的平台上買到但只有新幣計價的選擇而且績效很差 大盤漲 基金不見得會...,Re: 有人投資過越南基金嗎?,2011-01-01 22:37:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","["", 利安, 越南, 基金, 可以, 在, 先鋒, 投顧, 的, 平台, 上, 買到, 但...","["", 利安, 越南, 基金, 先鋒, 投顧, 平台, 買到, 新幣, 計價, 選擇, 績效...",2011-01-01
2,11,如果出口的力道逐漸復甦科技股應該還是會是重頭戲吧,Re: 今年的台股基金投資方向,2011-01-03 16:14:00,king,"""964""",NaN,天使人,"""2010-12-10 09:37""","["", 如果, 出口, 的, 力道, 逐漸, 復甦, 科技股, 應該, 還是, 會, 是, ...","["", 出口, 力道, 逐漸, 復甦, 科技股, 應該, 會, 重頭戲, ""]",2011-01-03
3,11,發問的大大能否請問你傳產賺到錢的是哪一檔基金阿?另請問強基金大大推薦統一大滿貫/群益葛萊美/...,Re: 今年的台股基金投資方向,2011-01-03 16:27:00,ryoma,"""634""",NaN,名人堂成員 兼 分版版主,"""2011-01-03 16:18""","["", 發問, 的, 大大, 能否, 請問, 你, 傳產, 賺到, 錢, 的, 是, 哪, ...","["", 發問, 大大, 請問, 傳產, 賺到, 錢, 檔, 基金, 請問, 強, 基金, 大...",2011-01-03
4,11,@chenyu今年強基金很看好台股上萬點推升力道來自國外的熱錢炒作的題材當然就是ECFA連香...,Re: 今年的台股基金投資方向,2011-01-03 17:47:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","[""@chenyu, 今年, 強, 基金, 很, 看好, 台股, 上萬, 點, 推升, 力道...","[""@chenyu, 今年, 強, 基金, 看好, 台股, 上萬, 點, 推升, 力道, 國...",2011-01-03


In [ ]:
y.head()

,ym,redemption,delta,signal
0,2014-11-30,0.039739,NaN,False
1,2014-12-31,0.041614,0.001875,True
2,2015-01-31,0.041046,-0.000568,False
3,2015-02-28,0.041046,0.000000,False
4,2015-03-31,0.081560,0.040514,True


# 加入stopwords字典

In [ ]:
word_sentence_list = df['seg']

In [ ]:
stopWords=[]
with open('../content/drive/My Drive/Nomura/stopword.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopWords.append(data)
        remainder_sentence = []

stopWords.append('\n')
apndix = ['(', '61', '.231', '.', '200', '.', '192', ' ', '臺灣', ')', ' ', '05', '/', '01', '/', '2020', ' ', '15', ':', '34', ':', '14\n',  '--\n', '※ '  ]
for i in apndix:
    stopWords.append(i)


In [ ]:
for i in word_sentence_list:
    remainderWords = list(filter(lambda a: a not in stopWords and a != '\n', i))
    remainder_sentence.append(remainderWords)


In [ ]:
print(remainder_sentence[0:2])

[['"', '檔', '利安', '越南', '基金', '喔', '試試', '"'], ['"', '利安', '越南', '基金', '先鋒', '投顧', '平台', '買到', '新幣', '計價', '選擇', '績效', '差', '\u3000', '大盤', '漲', '\u3000', '基金', '不見得', '會', '動強', '基金', '想', '投資', '越南', '找', '對應', '基金', '挺', '可惜', '"']]


In [ ]:
stoppedContend=[]
for sentence in remainder_sentence:
  stoppedContend.append(''.join(sentence))

In [ ]:
print(stoppedContend[0:2])

['"檔利安越南基金喔試試"', '"利安越南基金先鋒投顧平台買到新幣計價選擇績效差\u3000大盤漲\u3000基金不見得會動強基金想投資越南找對應基金挺可惜"']


# 整理資料

In [ ]:
df['stopped_content']= stoppedContend

In [ ]:
df.head()

,article_index,content,title,time,author,articles_no_author,from,author_identity,author_enroll_time,word_sentence_list,seg,date,stopped_content
0,10,有一檔利安越南基金喔可以試試看,Re: 有人投資過越南基金嗎?,2011-01-01 22:34:00,8d8d,"""725""",NaN,天使人,"""2010-12-10 09:48""","["", 有, 一, 檔, 利安, 越南, 基金, 喔, 可以, 試試, 看, ""]","["", 檔, 利安, 越南, 基金, 喔, 試試, ""]",2011-01-01,"""檔利安越南基金喔試試"""
1,10,利安越南基金可以在先鋒投顧的平台上買到但只有新幣計價的選擇而且績效很差 大盤漲 基金不見得會...,Re: 有人投資過越南基金嗎?,2011-01-01 22:37:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","["", 利安, 越南, 基金, 可以, 在, 先鋒, 投顧, 的, 平台, 上, 買到, 但...","["", 利安, 越南, 基金, 先鋒, 投顧, 平台, 買到, 新幣, 計價, 選擇, 績效...",2011-01-01,"""利安越南基金先鋒投顧平台買到新幣計價選擇績效差 大盤漲 基金不見得會動強基金想投資越南找對..."
2,11,如果出口的力道逐漸復甦科技股應該還是會是重頭戲吧,Re: 今年的台股基金投資方向,2011-01-03 16:14:00,king,"""964""",NaN,天使人,"""2010-12-10 09:37""","["", 如果, 出口, 的, 力道, 逐漸, 復甦, 科技股, 應該, 還是, 會, 是, ...","["", 出口, 力道, 逐漸, 復甦, 科技股, 應該, 會, 重頭戲, ""]",2011-01-03,"""出口力道逐漸復甦科技股應該會重頭戲"""
3,11,發問的大大能否請問你傳產賺到錢的是哪一檔基金阿?另請問強基金大大推薦統一大滿貫/群益葛萊美/...,Re: 今年的台股基金投資方向,2011-01-03 16:27:00,ryoma,"""634""",NaN,名人堂成員 兼 分版版主,"""2011-01-03 16:18""","["", 發問, 的, 大大, 能否, 請問, 你, 傳產, 賺到, 錢, 的, 是, 哪, ...","["", 發問, 大大, 請問, 傳產, 賺到, 錢, 檔, 基金, 請問, 強, 基金, 大...",2011-01-03,"""發問大大請問傳產賺到錢檔基金請問強基金大大推薦統一大滿貫群益葛萊美保德信高成長原因"""
4,11,@chenyu今年強基金很看好台股上萬點推升力道來自國外的熱錢炒作的題材當然就是ECFA連香...,Re: 今年的台股基金投資方向,2011-01-03 17:47:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","[""@chenyu, 今年, 強, 基金, 很, 看好, 台股, 上萬, 點, 推升, 力道...","[""@chenyu, 今年, 強, 基金, 看好, 台股, 上萬, 點, 推升, 力道, 國...",2011-01-03,"""@chenyu今年強基金看好台股上萬點推升力道國外熱錢炒作題材ECFA香港股神曹仁超說EC..."


In [ ]:
from pandas.tseries.offsets import MonthEnd
df['time'] = pd.to_datetime(df['time'])
df['EndOfMonth'] = pd.to_datetime(df['time'], format="%Y%m") + MonthEnd(1)

In [ ]:
df.head()

,article_index,content,title,time,author,articles_no_author,from,author_identity,author_enroll_time,word_sentence_list,seg,date,stopped_content,EndOfMonth
0,10,有一檔利安越南基金喔可以試試看,Re: 有人投資過越南基金嗎?,2011-01-01 22:34:00,8d8d,"""725""",NaN,天使人,"""2010-12-10 09:48""","["", 有, 一, 檔, 利安, 越南, 基金, 喔, 可以, 試試, 看, ""]","["", 檔, 利安, 越南, 基金, 喔, 試試, ""]",2011-01-01,"""檔利安越南基金喔試試""",2011-01-31 22:34:00
1,10,利安越南基金可以在先鋒投顧的平台上買到但只有新幣計價的選擇而且績效很差 大盤漲 基金不見得會...,Re: 有人投資過越南基金嗎?,2011-01-01 22:37:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","["", 利安, 越南, 基金, 可以, 在, 先鋒, 投顧, 的, 平台, 上, 買到, 但...","["", 利安, 越南, 基金, 先鋒, 投顧, 平台, 買到, 新幣, 計價, 選擇, 績效...",2011-01-01,"""利安越南基金先鋒投顧平台買到新幣計價選擇績效差 大盤漲 基金不見得會動強基金想投資越南找對...",2011-01-31 22:37:00
2,11,如果出口的力道逐漸復甦科技股應該還是會是重頭戲吧,Re: 今年的台股基金投資方向,2011-01-03 16:14:00,king,"""964""",NaN,天使人,"""2010-12-10 09:37""","["", 如果, 出口, 的, 力道, 逐漸, 復甦, 科技股, 應該, 還是, 會, 是, ...","["", 出口, 力道, 逐漸, 復甦, 科技股, 應該, 會, 重頭戲, ""]",2011-01-03,"""出口力道逐漸復甦科技股應該會重頭戲""",2011-01-31 16:14:00
3,11,發問的大大能否請問你傳產賺到錢的是哪一檔基金阿?另請問強基金大大推薦統一大滿貫/群益葛萊美/...,Re: 今年的台股基金投資方向,2011-01-03 16:27:00,ryoma,"""634""",NaN,名人堂成員 兼 分版版主,"""2011-01-03 16:18""","["", 發問, 的, 大大, 能否, 請問, 你, 傳產, 賺到, 錢, 的, 是, 哪, ...","["", 發問, 大大, 請問, 傳產, 賺到, 錢, 檔, 基金, 請問, 強, 基金, 大...",2011-01-03,"""發問大大請問傳產賺到錢檔基金請問強基金大大推薦統一大滿貫群益葛萊美保德信高成長原因""",2011-01-31 16:27:00
4,11,@chenyu今年強基金很看好台股上萬點推升力道來自國外的熱錢炒作的題材當然就是ECFA連香...,Re: 今年的台股基金投資方向,2011-01-03 17:47:00,fundhot,"""7483""","""我是強大""",管理員,"""2010-12-10 09:47""","[""@chenyu, 今年, 強, 基金, 很, 看好, 台股, 上萬, 點, 推升, 力道...","[""@chenyu, 今年, 強, 基金, 看好, 台股, 上萬, 點, 推升, 力道, 國...",2011-01-03,"""@chenyu今年強基金看好台股上萬點推升力道國外熱錢炒作題材ECFA香港股神曹仁超說EC...",2011-01-31 17:47:00


# 資料合併

In [ ]:
df['EndOfMonth'] = df['EndOfMonth'].apply(lambda x:x.date())

In [ ]:
y.head()

,ym,redemption,delta,signal
0,2014-11-30,0.039739,NaN,False
1,2014-12-31,0.041614,0.001875,True
2,2015-01-31,0.041046,-0.000568,False
3,2015-02-28,0.041046,0.000000,False
4,2015-03-31,0.081560,0.040514,True


In [ ]:
y['ym'] = pd.to_datetime(y['ym']).apply(lambda x:x.date())

In [ ]:
data = df.merge(y,   left_on='EndOfMonth',right_on='ym')

In [ ]:
from keras_bert import load_trained_model_from_checkpoint

# BERT 透過 Subword 的方式讓辭典長度最大就是 30522
token_dict = {}
with open(vocab_path, 'r', encoding='utf8') as f:
    for line in f.readlines():
        token = line.strip()
        token_dict[token] = len(token_dict)
print("辭典長度:", len(token_dict))


model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=False,
    trainable=False,
    seq_len=SEQ_LEN,
)

Using TensorFlow backend.


辭典長度: 21128


In [ ]:
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)

# 整理成可跑型態

In [ ]:
data.head()

,article_index,content,title,time,author,articles_no_author,from,author_identity,author_enroll_time,word_sentence_list,seg,date,stopped_content,EndOfMonth,ym,redemption,delta,signal
0,177,11:00 新台幣貶2.82角 報32.0元兌1美元鉅亨網新聞中心 2015-01-05 ...,Re: 關於匯率避險,2015-01-05 11:58:00,baonamy,"""823""",NaN,名人堂成員,"""2011-01-16 23:00""","["", 11:00, , 新台幣, 貶, 2.82, 角, , 報, 32.0, 元, ...","["", 11:00, , 新台幣, 貶, 2.82, 角, , 報, 32.0, 元, ...",2015-01-05,"""11:00新台幣貶2.82角報32.0元兌美元鉅亨網新聞中心 2015-01-05 1...",2015-01-31,2015-01-31,0.041046,-0.000568,False
1,177,有位單筆操作比我厲害好幾倍的大大問我對上証走勢的看法家庭主婦不敢拿水果刀當大刀耍倒是有個圖意...,Re: 關於匯率避險,2015-01-15 23:35:00,Ivy,"""2373""",NaN,名人堂成員,"""2012-10-08 19:52""","["", 有, 位, 單, 筆, 操作, 比, 我, 厲害, 好幾, 倍, 的, 大大, 問,...","["", 位, 單, 筆, 操作, 厲害, 好幾, 倍, 大大, 問, 上証, 走勢, 看法,...",2015-01-15,"""位單筆操作厲害好幾倍大大問上証走勢看法家庭主婦敢水果刀大刀耍倒是圖意圖透露給一點兒訊息""",2015-01-31,2015-01-31,0.041046,-0.000568,False
2,177,不知道匯率目前該存啥比較好 現在還是主要存美元 等歐元看真ㄉ有沒有1:1以下可以撿(簡單又保...,Re: 關於匯率避險,2015-01-16 06:01:00,apple,"""81""",NaN,註冊會員,"""2012-04-02 22:40""","["", 不, 知道, 匯率, 目前, 該, 存, 啥, 比較, 好, , 現在, 還是, ...","["", 知道, 匯率, 目前, 存, 比較, , 現在, 主要, 存, 美元, , 歐元...",2015-01-16,"""知道匯率目前存比較現在主要存美元歐元真ㄉ沒有1:1以下撿簡單保比較安心""",2015-01-31,2015-01-31,0.041046,-0.000568,False
3,177,感謝ivy姐回覆，若依匯率KD來看，中線準備看升，短線準備看貶，基於中線保護短線，等短線回檔...,Re: 關於匯率避險,2015-01-16 10:11:00,simonks,"""960""",NaN,名人堂成員,"""2013-02-24 21:01""","["", 感謝, ivy, 姐, 回覆, ，, 若, 依, 匯率, KD, 來, 看, ，, ...","["", 感謝, ivy, 姐, 回覆, 匯率, KD, 中線, 準備, 看升, 短線, 準備...",2015-01-16,"""感謝ivy姐回覆匯率KD中線準備看升短線準備看貶中線保護短線短線回檔日kd死叉適時介入上證...",2015-01-31,2015-01-31,0.041046,-0.000568,False
4,177,"sorry沒注意到您有回這則個人看法是上證仍有機會受惠於年前逆回購及節慶消費價格是真實的,再...",Re: 關於匯率避險,2015-01-28 11:27:00,Ivy,"""2373""",NaN,名人堂成員,"""2012-10-08 19:52""","[""sorry, 沒, 注意到, 您, 有, 回, 這, 則, 個人, 看法, 是, 上證,...","[""sorry, 沒, 注意到, 回, 個人, 看法, 上證, 機會, 受惠, 年前, 逆,...",2015-01-28,"""sorry沒注意到回個人看法上證機會受惠年前逆回購節慶消費價格真實獻新圖美元人民幣美元台幣...",2015-01-31,2015-01-31,0.041046,-0.000568,False


In [ ]:
index_array = (data['author_identity']=="名人堂成員") |  (data['author_identity']=="管理員")

In [ ]:
# index_array = data['type'].apply(lambda x: x not in (['新聞', '標的'])) 
# X=data[index_array ]['content'].iloc[0:100].apply(lambda x:tokenizer.encode(x, max_len=SEQ_LEN))
traindata = data[index_array]
indices=[]
y=[]
for content, label in zip(traindata['stopped_content'],traindata['signal'] ) :
  ids, segments = tokenizer.encode(content, max_len=SEQ_LEN)
  indices.append(ids)

  y.append(label)

# Y=data[index_array ]['label'].iloc[0:100]

In [ ]:
X = [indices, np.zeros_like(indices)]
# X

In [ ]:
# import numpy as np
# Xdata = [a, np.zeros_like(a)]

In [ ]:
# en, seg = tokenizer.encode(a[0])
# print("[Encoding]:", en)
# print("[Segement]:", seg)
# de = tokenizer.decode(a[0])
# print("[Decode]:", de)

In [ ]:
# data['stopped_content'][0]

In [ ]:
# # 準備資料流
# import glob
# from tqdm import tqdm
# import numpy as np

# def load_data(path):
#     global tokenizer
#     indices, sentiments = [], []
#     for folder, sentiment in (('neg', 0), ('pos', 1)):
#         pat = glob.glob("{}/{}/*".format(path, folder))
#         for fn in tqdm(iter(pat), total=len(pat)):
#             with open(fn, 'r', encoding="utf-8") as f:
#                   text = f.read()
#             ids, segments = tokenizer.encode(text, max_len=SEQ_LEN)
#             indices.append(ids)
#             sentiments.append(sentiment)
#     items = list(zip(indices, sentiments))
#     np.random.shuffle(items)
#     indices = np.array([i for i, s in items])
#     sentiments = np.array([s for i, s in items])

#     return [indices, np.zeros_like(indices)], np.array(sentiments)

# # 讀取資料集
# train_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'train')
# test_path = os.path.join(os.path.dirname(dataset), 'aclImdb', 'test')

# train_x, train_y = load_data(train_path)
# test_x, test_y = load_data(test_path)

In [ ]:
x

<tf.Tensor 'simple_rnn_1/strided_slice_2:0' shape=(None, 64) dtype=float32>

# 跑模型

In [ ]:
import keras
from keras.layers import SimpleRNN
inputs = model.inputs[:2]
x = SimpleRNN(64)(model.output)
outputs = keras.layers.Dense(units=2, activation='softmax')(x)
model = keras.models.Model(inputs, outputs)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=['sparse_categorical_accuracy']
)

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [ ]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

import tensorflow as tf
from keras_bert import get_custom_objects

TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

KeyError: ignored

In [ ]:
model.fit(
    X,
    y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.2
)

Train on 7103 samples, validate on 1776 samples
Epoch 1/5
7103/7103 [==============================] - 845s 119ms/step - loss: 0.6976 - sparse_categorical_accuracy: 0.5216 - val_loss: 0.7183 - val_sparse_categorical_accuracy: 0.4657
Epoch 2/5
7103/7103 [==============================] - 840s 118ms/step - loss: 0.6873 - sparse_categorical_accuracy: 0.5433 - val_loss: 0.7035 - val_sparse_categorical_accuracy: 0.4690
Epoch 3/5
7103/7103 [==============================] - 840s 118ms/step - loss: 0.6779 - sparse_categorical_accuracy: 0.5643 - val_loss: 0.6953 - val_sparse_categorical_accuracy: 0.5676
Epoch 4/5
7103/7103 [==============================] - 836s 118ms/step - loss: 0.6733 - sparse_categorical_accuracy: 0.5709 - val_loss: 0.6954 - val_sparse_categorical_accuracy: 0.5619
Epoch 5/5
7103/7103 [==============================] - 836s 118ms/step - loss: 0.6580 - sparse_categorical_accuracy: 0.6100 - val_loss: 0.7013 - val_sparse_categorical_accuracy: 0.5518


In [ ]:
model.save('bert_model_strongfund.h5') 


In [ ]:
model.save('../content/drive/My Drive/Nomura/bert_model_strongfund.h5')

# load model

In [ ]:
model = load_model('my_model.h5')
print('test after load: ', model.predict(X_test[0:2]))

In [ ]:
# model.evaluate(test_x, test_y)